# User Input + StateFlow GroupChat: Build Workflows through State-Oriented Actions that can intelligently include the user in the process

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. In this notebook, we introduce how to use groupchat to build workflows with AutoGen agents from a state-oriented perspective and allow for user input during the process.

This notebook is an expansion of [StateFlow: Build Workflows through State-Oriented Actions](https://microsoft.github.io/autogen/docs/notebooks/agentchat_groupchat_stateflow/) to allow for user input during the process. 


````{=mdx}
:::info Requirements
Install `autogen-agentchat`:
```bash
pip install autogen-agentchat~=0.2
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "tags": ["gpt-4o-2024-05-13"],
    },
)

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````
## Workflow: Arxiv Research
We define the following agents:
- Planner: Create a plan.
- Admin: Human in the loop to provide optional feedback.
- Engineer: Retrieve papers from the internet by writing code.
- Executor: Execute the code.
- Scientist: Read the papers and write a summary.

The pipeline is the following:
0. Input: a str prompt -> a research topic from Arxiv
1. Plan: The planner will create a plan based on the prompt and decide if it would like user input. 
2. Optional_plan_input: If they don't, workflow continues, if they do, the Admin will be asked for input before returning to step 1.
3. Code: The engineer will write code to retrieve papers from the internet. 
4. Optional_code_input: If the engineer is stuck or doesn't know how to solve the task, they can ask the user for input before returning to step 3. 
5. Execution: The code will be executed by executor. If the code execution fails, we go back to step 3. 
6. Summary: If the code is executed successfully, the scientist will read the papers and write a summary. They can choose to ask the user for input. 
7. Optional_summary_input: Once the summary is received, if user input was asked, the Admin will be asked for input, and we go back to step 5. 

In [3]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""You are the Planner for a research workflow. The user will provide a task, and your job is to suggest a plan. 
The plan can involve an engineer who can write code and a scientist who doesn't write code. The engineer always acts first, and then the scientist. 
Do not suggest recursive plans where agents need to interact multiple times. Keep it simple. 
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist. 
If you need to ask the user for more information, you can do so by including ´ASK_USER:´ in your response. If you do, use the answer and respond with a complete plan. 
""",
    llm_config=gpt4_config,
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interacts with the planner and scientist if asked.",
    code_execution_config=False,
    default_auto_reply="No user input received"
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""You are an Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
If you can't solve the task, you can ask the user for more information by including ´ASK_USER:´ in your response. If you do, use the answer and respond with a complete code solution.
""",
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""You are a Scientist. You are part of a research workflow, which has been planned. 
Follow the plan the Planner suggested, and provide a summary of the information retrieved. You don't write code. 
If you need to ask the user for more information, you can do so by including ´ASK_USER:´ in your response. If you do, use the answer and respond with a complete summary.""",
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

from typing import Dict, List
from autogen import Agent


def custom_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages

    if len(messages) <= 1:
        return planner
    
    if last_speaker is user_proxy:
        if messages[-2]["name"] == "Planner":
            # If it is the planning stage, let the planner continue
            return planner
        elif messages[-2]["name"] == "Scientist":
            # If the last message is from the scientist, let the scientist continue
            return scientist
        elif messages[-2]["name"] == "Engineer":
            # If the last message is from the engineer, let the engineer continue
            return engineer
        
    elif "ASK_USER:" in messages[-1]["content"] and last_speaker is not executor:
        # If someone asks for user input, let the user speak
        return user_proxy

    elif last_speaker is planner:
        # If the last message is from the planner, we continue to the engineer because they didn't ask for user input
        return engineer

    elif last_speaker is engineer:
        if "```python" in messages[-1]["content"]:
            # If the last message is a python code block, let the executor speak
            return executor
        else:
            # Otherwise, let the engineer continue -> This can be confusing, so adding a message here to clarify the issue would be better
            return engineer

    elif last_speaker is executor:
        if "exitcode: 1" in messages[-1]["content"]:
            # If the last message indicates an error, let the engineer improve the code
            return engineer
        else:
            # Otherwise, let the scientist speak
            return scientist

    elif last_speaker is scientist:
        # If the last message is from the scientist, we end because they didn't request user input
        return None
    else:
        return "random"


groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor],
    messages=[],
    max_round=5,
    speaker_selection_method=custom_speaker_selection_func,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [4]:
chat_result = user_proxy.initiate_chat(
    manager, 
    message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software.",
    max_turns=20,
)

Admin (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Planner (to chat_manager):

### Plan

1. **Engineer Step:**
   - The engineer will write a script to search for the latest papers about GPT-4 on arXiv. This script will use the arXiv API to fetch the most recent papers related to GPT-4.
   - The engineer will then extract the relevant information from the fetched papers, such as the title, authors, abstract, and publication date.
   - The engineer will provide a summary of the latest paper found, including a link to the full paper.

2. **Scientist Step:**
   - The scientist will read the summary and the full paper provided by the engineer.
   - The scientist will analyze the paper to identify potential applications of GPT-4 in software.
   - The scientist will document these potential applications in a detailed report.

### Execution

1. **E